In [1]:
import numpy as np  # For matrix operations and numerical processing
import pandas as pd  # For munging tabular data
import sklearn as sk  # For access to a variety of machine learning models
import matplotlib.pyplot as plt  # For charts and visualizations
from time import gmtime, strftime  # For labeling SageMaker models, endpoints, etc.
import sys  # For writing outputs to notebook

In [2]:
import catboost as cb
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [3]:
def first_look(df):
    """Display basic information about the dataframe"""
    df.columns = map(str.lower, df.columns)
    
    display(df.info())
    display(df.head(5))
    display(df.describe(include='all'))
    display('Доля пустых строк:', df.isna().mean()[df.isna().mean().values > 0])
    display('Количество дублирующихся строк:', df.duplicated().sum())
    

In [4]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [5]:
train = pd.read_csv("train_dataset_hackathon_mkb.csv",encoding='cp1251', sep=';')

In [6]:
first_look(train)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17891 entries, 0 to 17890
Columns: 124 entries, id_contract to citizenship_name
dtypes: float64(108), int64(4), object(12)
memory usage: 16.9+ MB


None

,id_contract,id_client,sign_date,ip_flag,target,f1100,f1110,f1150,f1160,f1170,...,winnernumber_95_ever,signednumber_95_ever,sum_95_ever,flag_disqualification,count_change_year,count_change_ever,birthdate,age,sex_name,citizenship_name
0,1,1847,01JAN2018:00:00:00,0,0,1.298961e+09,2154000.0,1.125573e+09,NaN,150010000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,4650,01JAN2018:00:00:00,1,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,4770,01JAN2018:00:00:00,0,0,7.337400e+07,NaN,7.337400e+07,NaN,NaN,...,169.0,168.0,18351739.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN
3,4,12237,01JAN2018:00:00:00,0,0,1.937488e+09,122828000.0,6.103280e+08,NaN,809426000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,9988,01JAN2018:00:00:00,1,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,id_contract,id_client,sign_date,ip_flag,target,f1100,f1110,f1150,f1160,f1170,...,winnernumber_95_ever,signednumber_95_ever,sum_95_ever,flag_disqualification,count_change_year,count_change_ever,birthdate,age,sex_name,citizenship_name
count,17891.000000,17891.000000,17891,17891.000000,17891.000000,6.936000e+03,1.420000e+03,6.341000e+03,3.460000e+02,1.499000e+03,...,6784.000000,6784.000000,6.784000e+03,5.0,239.000000,1509.000000,1419,1419.000000,2168,2128
unique,NaN,NaN,439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,844,NaN,2,4
top,NaN,NaN,09JAN2019:00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,05MAR1987:00:00:00,NaN,мужской,Российская Федерация
freq,NaN,NaN,198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,125,NaN,1265,2124
mean,8946.000000,6630.652786,NaN,0.195629,0.413001,3.184582e+09,1.136357e+08,2.287078e+09,4.981780e+07,3.389027e+09,...,354.557341,323.424823,2.544790e+08,1.0,1.502092,2.325381,NaN,41.916843,NaN,NaN
std,5164.831169,3659.132885,NaN,0.396695,0.492387,3.062184e+10,9.413832e+08,2.016291e+10,2.561433e+08,2.076766e+10,...,1339.773863,1258.815709,1.140965e+09,0.0,1.511509,2.316937,NaN,10.769157,NaN,NaN
min,1.000000,1.000000,NaN,0.000000,0.000000,-1.820000e+05,1.000000e+03,1.000000e+03,-1.175000e+06,1.000000e+03,...,0.000000,0.000000,0.000000e+00,1.0,1.000000,1.000000,NaN,21.000000,NaN,NaN
25%,4473.500000,3452.000000,NaN,0.000000,0.000000,4.335000e+05,7.300000e+04,7.180000e+05,3.694000e+06,1.130000e+05,...,17.000000,12.000000,4.296130e+06,1.0,1.000000,1.000000,NaN,32.000000,NaN,NaN
50%,8946.000000,6560.000000,NaN,0.000000,0.000000,4.308000e+06,2.760000e+05,4.984000e+06,3.694000e+06,7.239000e+06,...,75.000000,58.000000,1.865707e+07,1.0,1.000000,2.000000,NaN,40.000000,NaN,NaN
75%,13418.500000,10080.000000,NaN,0.000000,1.000000,6.661650e+07,4.409500e+06,5.594400e+07,1.195650e+07,1.485000e+08,...,219.250000,176.250000,9.359097e+07,1.0,2.000000,2.000000,NaN,49.000000,NaN,NaN


'Доля пустых строк:'

f1100                0.612319
f1110                0.920630
f1150                0.645576
f1160                0.980661
f1170                0.916215
                       ...   
count_change_ever    0.915656
birthdate            0.920686
age                  0.920686
sex_name             0.878822
citizenship_name     0.881058
Length: 119, dtype: float64

'Количество дублирующихся строк:'

0

In [7]:
train.isna().mean()[train.isna().mean().values > 0.9]

f1110                    0.920630
f1160                    0.980661
f1170                    0.916215
f1190                    0.915041
f1320                    0.994578
f1350                    0.946454
f1360                    0.923481
f1450                    0.959141
f1530                    0.957968
f1550                    0.949639
f2310                    0.983176
flag_disqualification    0.999721
count_change_year        0.986641
count_change_ever        0.915656
birthdate                0.920686
age                      0.920686
dtype: float64

In [8]:
train.isna().mean()[train.isna().mean().values == 0]

id_contract    0.0
id_client      0.0
sign_date      0.0
ip_flag        0.0
target         0.0
dtype: float64

Столбцы birthdate, age, sex_name, citizenship_name удалим из за большого количества пропусков, остальные пропуски заполним 0.

In [9]:
train.drop(['birthdate', 'age', 'sex_name', 'citizenship_name','datefirstreg','taxreg_regdate', 'taxregpay_regdate'], axis = 1, inplace=True)

In [10]:
train['sign_date'] = pd.to_datetime(train['sign_date'], format='%d%b%Y:%H:%M:%S')

In [11]:
# train[['sign_date','datefirstreg','taxreg_regdate', 'taxregpay_regdate']] = train[['sign_date','datefirstreg','taxreg_regdate', 'taxregpay_regdate']].apply(pd.to_datetime, format='%d%b%Y:%H:%M:%S')

In [12]:
# train[['sign_date','datefirstreg','taxreg_regdate', 'taxregpay_regdate']].isna().sum()

In [13]:
train = train.fillna(0)

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17891 entries, 0 to 17890
Columns: 117 entries, id_contract to count_change_ever
dtypes: datetime64[ns](1), float64(107), int64(4), object(5)
memory usage: 16.0+ MB


In [15]:
test = pd.read_csv("test_dataset_hackathon_mkb.csv",encoding='cp1251', sep=';')

In [16]:
first_look(test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7330 entries, 0 to 7329
Columns: 123 entries, id_contract to citizenship_name
dtypes: float64(108), int64(3), object(12)
memory usage: 6.9+ MB


None

,id_contract,id_client,sign_date,ip_flag,f1100,f1110,f1150,f1160,f1170,f1180,...,winnernumber_95_ever,signednumber_95_ever,sum_95_ever,flag_disqualification,count_change_year,count_change_ever,birthdate,age,sex_name,citizenship_name
0,17892,3620,08APR2019:00:00:00,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17893,4101,08APR2019:00:00:00,0,1906000.0,NaN,1906000.0,NaN,NaN,NaN,...,7439.0,7100.0,1.187411e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17894,9589,08APR2019:00:00:00,0,147000.0,NaN,147000.0,NaN,NaN,NaN,...,362.0,344.0,9.836260e+07,NaN,1.0,3.0,NaN,NaN,NaN,NaN
3,17895,11546,08APR2019:00:00:00,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17896,12558,08APR2019:00:00:00,0,26000.0,NaN,NaN,NaN,26000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


,id_contract,id_client,sign_date,ip_flag,f1100,f1110,f1150,f1160,f1170,f1180,...,winnernumber_95_ever,signednumber_95_ever,sum_95_ever,flag_disqualification,count_change_year,count_change_ever,birthdate,age,sex_name,citizenship_name
count,7330.000000,7330.000000,7330,7330.000000,2.490000e+03,4.580000e+02,2.321000e+03,7.900000e+01,4.860000e+02,6.790000e+02,...,2306.000000,2306.000000,2.306000e+03,5.0,79.000000,487.000000,615,615.000000,1434,1431
unique,NaN,NaN,248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,485,NaN,2,4
top,NaN,NaN,27MAY2019:00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,01SEP1986:00:00:00,NaN,мужской,Российская Федерация
freq,NaN,NaN,112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,21,NaN,858,1427
mean,21556.500000,6625.300819,NaN,0.243247,1.173428e+09,6.875055e+07,7.290510e+08,4.316418e+07,1.604195e+09,9.530686e+07,...,232.701214,221.149610,2.288399e+08,1.0,1.139241,2.433265,NaN,43.427642,NaN,NaN
std,2116.133069,3580.337029,NaN,0.429072,1.736408e+10,6.453434e+08,9.419810e+09,2.633641e+08,1.398350e+10,4.181462e+08,...,710.966033,691.959633,1.430015e+09,0.0,0.635170,2.371635,NaN,11.080438,NaN,NaN
min,17892.000000,4.000000,NaN,0.000000,4.740000e+02,1.000000e+03,4.740000e+02,6.000000e+03,1.000000e+03,-1.830000e+05,...,0.000000,0.000000,0.000000e+00,1.0,1.000000,1.000000,NaN,23.000000,NaN,NaN
25%,19724.250000,3778.250000,NaN,0.000000,3.652500e+05,6.150000e+04,4.260000e+05,1.167000e+06,6.200000e+04,1.720000e+05,...,10.000000,9.000000,3.528744e+06,1.0,1.000000,1.000000,NaN,34.000000,NaN,NaN
50%,21556.500000,6758.500000,NaN,0.000000,3.014000e+06,4.360000e+05,3.039000e+06,3.592000e+06,8.805000e+05,1.277000e+06,...,50.000000,44.000000,1.813208e+07,1.0,1.000000,1.000000,NaN,42.000000,NaN,NaN
75%,23388.750000,9801.250000,NaN,0.000000,3.051525e+07,2.015425e+07,2.042800e+07,8.238000e+06,6.500200e+07,1.485150e+07,...,153.000000,146.000000,7.183648e+07,1.0,1.000000,3.000000,NaN,51.000000,NaN,NaN


'Доля пустых строк:'

f1100                0.660300
f1110                0.937517
f1150                0.683356
f1160                0.989222
f1170                0.933697
                       ...   
count_change_ever    0.933561
birthdate            0.916098
age                  0.916098
sex_name             0.804366
citizenship_name     0.804775
Length: 119, dtype: float64

'Количество дублирующихся строк:'

0

In [17]:
test.drop(['birthdate', 'age', 'sex_name', 'citizenship_name','datefirstreg','taxreg_regdate', 'taxregpay_regdate'], axis = 1, inplace=True)

In [18]:
test['sign_date'] = pd.to_datetime(test['sign_date'], format='%d%b%Y:%H:%M:%S')

In [19]:
# test[['sign_date','datefirstreg','taxreg_regdate', 'taxregpay_regdate']] = test[['sign_date','datefirstreg','taxreg_regdate', 'taxregpay_regdate']].apply(pd.to_datetime, format='%d%b%Y:%H:%M:%S')

In [20]:
test = test.fillna(0)

In [21]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7330 entries, 0 to 7329
Columns: 116 entries, id_contract to count_change_ever
dtypes: datetime64[ns](1), float64(107), int64(3), object(5)
memory usage: 6.5+ MB


In [22]:
X = train.drop(['target'], axis = 1)
y = train.loc[:, 'target']

print(X.shape, y.shape)

(17891, 116) (17891,)


In [23]:
# Splitting X,y into train and validation data 
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state = 42)

In [24]:
model = CatBoostClassifier(iterations = 5000,
                          eval_metric='Accuracy',
                          od_wait=500,
                          task_type='GPU',
                         )
model.fit(x_train, y_train,
         eval_set=(x_valid, y_valid),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=64]="Частная собственность": Cannot convert 'b'\xd0\xa7\xd0\xb0\xd1\x81\xd1\x82\xd0\xbd\xd0\xb0\xd1\x8f \xd1\x81\xd0\xbe\xd0\xb1\xd1\x81\xd1\x82\xd0\xb2\xd0\xb5\xd0\xbd\xd0\xbd\xd0\xbe\xd1\x81\xd1\x82\xd1\x8c'' to float